# Hackaton Desafio Budokai

## Equipe 11

### notebook_v2: A ideia principal nesta versão é utilizar os dados mensais (média, desvio, max e min) como entrada no modelo de previsão da produção anual de soja.

## Bibliotecas:

In [1]:
#importanto bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, preprocessing

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler
from datetime import datetime

from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split

#### Lendo csv e removendo algumas colunas redundantes:

In [2]:
Y_train=pd.read_csv('shared/bases_budokai_ufpr/produtividade_soja_modelagem.csv')
df=pd.read_csv('shared/bases_budokai_ufpr/agroclimatology_budokai.csv')

In [3]:
Y_train.head()

,nivel,codigo_ibge,name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,MU,4100103,Abatiá,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,MU,4100459,Altamira do Paraná,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,MU,4100608,Alto Paraná,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,MU,4100707,Alto Piquiri,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,MU,4100509,Altônia,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


In [4]:
Y_train['nivel'].unique()

array(['MU'], dtype=object)

In [5]:
#Y_codigo=Y_train['codigo_ibge']

Y_train=Y_train.drop(columns=['nivel', 'name'])

In [6]:
Y_train.head()

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


### De acordo com as respostas esperadas temos:

In [7]:
codigo_saida=[4102000,4104303,4104428,4104808,4104907,4109401,4113205,4113700,4113734,4114005,4114401,4117701,4117909,4119608,4119905,4127007,4127403,4127502,4127700,4128005
]

In [8]:
dataset=pd.DataFrame(columns=df.keys())
#dataset.loc[dataset['codigo_ibge']==codigo_saida[j]]

for k in codigo_saida:
    dataset=pd.concat([df.loc[df['codigo_ibge']==k],dataset])

dataset=dataset.reset_index(drop=True)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131500 entries, 0 to 131499
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   data                 131500 non-null  object 
 1   codigo_ibge          131500 non-null  object 
 2   latitude             131500 non-null  float64
 3   longitude            131500 non-null  float64
 4   T2M_RANGE            131500 non-null  float64
 5   TS                   131500 non-null  float64
 6   T2MDEW               131500 non-null  float64
 7   T2MWET               131500 non-null  float64
 8   T2M_MAX              131500 non-null  float64
 9   T2M_MIN              131500 non-null  float64
 10  T2M                  131500 non-null  float64
 11  QV2M                 131500 non-null  float64
 12  RH2M                 131500 non-null  float64
 13  PRECTOTCORR          131500 non-null  float64
 14  PS                   131500 non-null  float64
 15  WS2M             

#### Transformando o codigo em inteiro novamente.

In [10]:
dataset['codigo_ibge']=list(map(int,dataset['codigo_ibge']))

#df.loc[df['codigo_ibge']==codigo_saida[0]]

### Vamos começar transformando os dados de data em 'datetime' para melhor agrupar em semanas, meses ou anos.

In [11]:
def t_data_day(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.day

def t_data_month(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.month

def t_data_year(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.year
    #return data.day, data.month, data.year

In [12]:
dataset['day']=list(map(t_data_day, dataset['data']))
dataset['month']=list(map(t_data_month, dataset['data']))
dataset['year']=list(map(t_data_year, dataset['data']))

In [13]:
dataset.head()

,data,codigo_ibge,latitude,longitude,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,day,month,year
0,20030101,4128005,-24.5393,-52.9865,7.99,26.27,23.23,24.83,30.21,22.21,...,30.59,42.56,133.74,165.12,18.27,0.56,2.95,1,1,2003
1,20030102,4128005,-24.5393,-52.9865,6.47,25.70,23.36,24.59,29.33,22.86,...,29.79,42.55,111.66,161.13,15.38,0.48,2.55,2,1,2003
2,20030103,4128005,-24.5393,-52.9865,6.27,24.75,22.78,23.78,28.03,21.76,...,29.66,42.53,89.88,161.48,12.85,0.41,2.12,3,1,2003
3,20030104,4128005,-24.5393,-52.9865,6.38,24.50,22.21,23.29,27.82,21.44,...,28.35,42.51,87.03,153.10,12.10,0.40,2.05,4,1,2003
4,20030105,4128005,-24.5393,-52.9865,8.43,25.62,22.50,23.95,29.68,21.24,...,29.04,42.49,52.05,156.25,7.56,0.24,1.24,5,1,2003


#### Uma feature importante é o valor da produção do ano passado, podemos tentar prever a produção de 2013 e criar uma feature depois.

In [14]:
dataset=dataset.drop(dataset.loc[dataset['year']==2003].index).reset_index(drop=True)

### Podemo implementar um sistema para previsões baseado na média dos valores mensais.

In [15]:
dataset=dataset.drop(columns=['data','latitude','longitude','day'])

#### Para cada codigo postal iremos fazer um subgrupo para cada ano e agrupar os meses pela média.

#### Depois concatenamos e criamos o novo dataset.

In [16]:
codigos=dataset['codigo_ibge'].unique()
anos=dataset['year'].unique()

datanew=pd.DataFrame(columns=dataset.keys())

for i in codigos:
    for j in anos:
        aux = dataset.loc[dataset['codigo_ibge']==i].loc[dataset['year']==j].groupby(by='month').agg('mean').reset_index()
        datanew=pd.concat([datanew,aux])
    
datanew=datanew.reset_index(drop=True)

In [17]:
datanew.head()

,codigo_ibge,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,QV2M,RH2M,...,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,month,year
0,4128005,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,...,23.842258,29.756452,42.040000,129.152258,157.907419,17.224516,0.526452,-29.573548,1,2004
1,4128005,12.955517,27.584483,18.363448,22.427931,33.331379,20.375172,26.492414,13.849310,64.810690,...,23.348966,28.156207,39.736207,126.227931,149.815517,16.812759,0.507586,2.653793,2,2004
2,4128005,13.622258,26.335806,17.173871,21.365806,32.728065,19.107419,25.557097,12.963548,64.225806,...,21.451613,25.462581,35.433226,113.812581,134.426774,15.015484,0.436452,-30.048387,3,2004
3,4128005,10.766667,24.591000,19.021667,21.618333,30.242667,19.473667,24.218667,14.357000,75.240000,...,16.357333,20.808667,29.943000,88.034000,111.137667,11.629667,0.323000,-31.676667,4,2004
4,4128005,9.320645,17.494516,14.863548,16.181613,22.614839,13.293871,17.502581,11.179355,85.648387,...,11.521935,17.273548,25.104194,60.481613,90.170645,7.898387,0.194516,-31.256452,5,2004


### Precisamos colocar os dados dos meses em uma unica linha para alimentar a entrada do modelo.

#### Criando a lista de indices:

In [18]:
col=list(datanew.keys())
col.remove('year')
col.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [19]:
lista_indices=[]

lista_indices.append('codigo_ibge')

for i in range(12):

    
    for j in col:
        
        aux=j+'_'+str(i+1)
        lista_indices.append(aux)
        
lista_indices.append('month')
lista_indices.append('year')

In [20]:
lista_indices

['codigo_ibge',
 'T2M_RANGE_1',
 'TS_1',
 'T2MDEW_1',
 'T2MWET_1',
 'T2M_MAX_1',
 'T2M_MIN_1',
 'T2M_1',
 'QV2M_1',
 'RH2M_1',
 'PRECTOTCORR_1',
 'PS_1',
 'WS2M_1',
 'WS2M_MAX_1',
 'WS2M_MIN_1',
 'WS2M_RANGE_1',
 'WS10M_1',
 'WS10M_MAX_1',
 'WS10M_MIN_1',
 'WS10M_RANGE_1',
 'WS50M_1',
 'WS50M_MAX_1',
 'WS50M_MIN_1',
 'WS50M_RANGE_1',
 'GWETTOP_1',
 'GWETROOT_1',
 'GWETPROF_1',
 'ALLSKY_SFC_SW_DWN_1',
 'CLRSKY_SFC_SW_DWN_1',
 'TOA_SW_DWN_1',
 'ALLSKY_SFC_PAR_TOT_1',
 'CLRSKY_SFC_PAR_TOT_1',
 'ALLSKY_SFC_UVA_1',
 'ALLSKY_SFC_UVB_1',
 'ALLSKY_SFC_UV_INDEX_1',
 'month_1',
 'T2M_RANGE_2',
 'TS_2',
 'T2MDEW_2',
 'T2MWET_2',
 'T2M_MAX_2',
 'T2M_MIN_2',
 'T2M_2',
 'QV2M_2',
 'RH2M_2',
 'PRECTOTCORR_2',
 'PS_2',
 'WS2M_2',
 'WS2M_MAX_2',
 'WS2M_MIN_2',
 'WS2M_RANGE_2',
 'WS10M_2',
 'WS10M_MAX_2',
 'WS10M_MIN_2',
 'WS10M_RANGE_2',
 'WS50M_2',
 'WS50M_MAX_2',
 'WS50M_MIN_2',
 'WS50M_RANGE_2',
 'GWETTOP_2',
 'GWETROOT_2',
 'GWETPROF_2',
 'ALLSKY_SFC_SW_DWN_2',
 'CLRSKY_SFC_SW_DWN_2',
 'TOA_SW_DWN_

#### Iremos criar um dataset menor para cada mês, mudar o nome dos indices para o indice_mes e depois juntar todos em um dataset com todos os indices ordenados por mês.

In [22]:

df_m=datanew[col].loc[datanew['month']==1].rename(columns=dict(zip(col[0:34],lista_indices[1:35]))).drop(columns='month')

a=35
b=69
for i in range(11):
        
    df_m2=datanew[col].loc[datanew['month']==i+2].rename(columns=dict(zip(col[0:34],lista_indices[a:b]))).drop(columns='month')
    a=b
    b=b+34
    df_m=pd.concat([df_m.reset_index(drop=True),df_m2.reset_index(drop=True)], axis=1)
    

In [23]:
df_m

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS2M_MIN_12,WS2M_RANGE_12,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,0.652581,0.662258,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,0.610000,0.614516,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,0.680000,0.675161,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,0.607097,0.613226,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,0.566774,0.573226,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,8.425484,23.434516,20.681290,22.045806,27.865161,19.438710,23.411290,16.423226,85.927097,9.002258,...,0.738065,0.729677,21.962903,30.644194,42.817742,119.750645,163.210645,16.173871,0.500000,2.630323
336,8.140968,23.840000,20.865484,22.313226,28.296452,20.155161,23.761935,16.457742,85.124194,7.655806,...,0.736129,0.728387,21.611290,30.383548,42.809032,118.041935,162.134516,15.931935,0.481290,-29.819677
337,7.663871,22.996129,20.359355,21.692258,27.200645,19.538387,23.025806,15.981613,86.016452,7.799032,...,0.667419,0.667097,25.597419,31.188065,42.804516,138.342258,165.951290,18.576452,0.563548,2.961290
338,9.545484,25.754839,21.086774,23.382581,30.743226,21.199677,25.675161,16.675484,78.064194,4.329032,...,0.770000,0.757419,22.388387,31.019032,42.801935,121.082258,164.555161,16.335484,0.492581,2.577742


#### Reinserindo ano e codigo do ibge:

In [24]:
df_m['codigo_ibge']=datanew['codigo_ibge'].loc[datanew['month']==1].reset_index(drop=True)
df_m['year']=datanew['year'].loc[datanew['month']==1].reset_index(drop=True)

In [25]:
df_m.head()

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12,codigo_ibge,year
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742,4128005,2004
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677,4128005,2005
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129,4128005,2006
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548,4128005,2007
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742,4128005,2008


In [26]:
X_train=df_m.loc[df_m['year']<2018]
X_test=df_m.loc[df_m['year']>2017]


#### Scalando os dados:

In [27]:
colunas_=list(X_train.keys())

colunas_.remove('year')
colunas_.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [28]:
len(colunas_)

408

In [29]:
X_train[colunas_]

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS2M_MIN_12,WS2M_RANGE_12,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12
0,12.313226,27.513548,19.130968,22.804839,32.921290,20.608387,26.477097,14.547742,67.569032,2.531935,...,0.652581,0.662258,23.674839,30.573548,42.462258,128.402581,162.105161,17.306452,0.508065,-29.687742
1,8.789677,26.006774,21.852903,23.737419,30.293226,21.501613,25.623871,17.136774,81.254516,7.784839,...,0.610000,0.614516,23.596129,30.226129,42.446129,128.496452,160.859677,17.292581,0.511613,2.659677
2,11.466774,28.138710,20.220000,23.727419,33.260645,21.793871,27.234516,15.484839,69.205484,4.431613,...,0.680000,0.675161,21.499032,29.592903,42.435806,117.794194,157.387097,15.792258,0.492258,2.596129
3,7.730000,25.683871,22.430323,23.973226,29.698710,21.968710,25.514194,17.737419,84.159032,8.712581,...,0.607097,0.613226,23.464194,30.246774,42.436129,127.693871,160.882258,17.193548,0.508065,2.643548
4,10.218710,26.340323,20.238710,22.941290,31.066452,20.848710,25.641935,15.544194,74.416774,3.609032,...,0.566774,0.573226,25.211290,30.488065,42.445484,136.012581,161.500968,18.207742,0.551935,-61.747742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,10.849355,23.446129,18.836774,21.040000,28.912258,18.062581,23.244839,14.600645,78.423548,4.669677,...,0.668065,0.665484,24.709355,30.882581,42.822581,134.274194,165.051613,18.018065,0.545806,2.874839
333,9.336129,24.036774,20.015484,22.099677,28.984194,19.649032,24.183226,15.638387,79.412581,6.101613,...,0.683548,0.680000,21.390645,30.376452,42.826452,116.979032,162.129032,15.787742,0.468065,2.419032
334,9.156774,24.720968,20.784194,22.766129,29.548065,20.391613,24.748710,16.382258,80.409355,4.708387,...,0.923871,0.901935,16.696774,29.344516,42.818065,92.785806,157.218387,12.595806,0.382258,1.980968
335,8.425484,23.434516,20.681290,22.045806,27.865161,19.438710,23.411290,16.423226,85.927097,9.002258,...,0.738065,0.729677,21.962903,30.644194,42.817742,119.750645,163.210645,16.173871,0.500000,2.630323


In [30]:
scaler=StandardScaler()


X_train_scaled=scaler.fit_transform(X_train[colunas_])
X_train_scaled=pd.DataFrame(X_train_scaled, columns=colunas_)


X_test_scaled=scaler.transform(X_test[colunas_])
X_test_scaled=pd.DataFrame(X_test_scaled, columns=colunas_)

### Precisamos reinserir o codigo_ibge e ano e resetar o indice.

In [31]:
X_train_scaled[['codigo_ibge','year']]=X_train[['codigo_ibge','year']].reset_index(drop=True)
X_test_scaled[['codigo_ibge','year']]=X_test[['codigo_ibge','year']].reset_index(drop=True)

In [32]:
X_train_scaled

,T2M_RANGE_1,TS_1,T2MDEW_1,T2MWET_1,T2M_MAX_1,T2M_MIN_1,T2M_1,QV2M_1,RH2M_1,PRECTOTCORR_1,...,WS10M_12,WS10M_MAX_12,WS10M_MIN_12,WS10M_RANGE_12,WS50M_12,WS50M_MAX_12,WS50M_MIN_12,WS50M_RANGE_12,codigo_ibge,year
0,1.975026,1.726639,-0.232708,0.709507,1.985233,0.695221,1.459704,-0.587431,-2.178146,-1.802781,...,0.585985,-0.098699,-1.300598,0.621381,-0.158816,0.626787,0.378930,-1.224826,4128005,2004
1,-0.440535,0.998187,1.349182,1.272883,0.737185,1.172972,1.006455,1.316658,0.272505,1.433995,...,0.553708,-0.548070,-1.360833,0.629091,-0.567499,0.617844,0.459727,0.496744,4128005,2005
2,1.394743,2.028874,0.400198,1.266842,2.146391,1.329289,1.862059,0.101751,-1.885108,-0.632222,...,-0.306271,-1.367119,-1.399384,-0.249892,-1.706963,-0.349442,0.019019,0.493362,4128005,2006
3,-1.166994,0.842078,1.684757,1.415335,0.454853,1.422804,0.948193,1.758399,0.792615,2.005659,...,0.499603,-0.521366,-1.398179,0.563174,-0.560089,0.553996,0.378930,0.495886,4128005,2007
4,0.539135,1.159441,0.411071,0.791938,1.104384,0.823760,1.016052,0.145404,-0.951925,-1.139087,...,1.216054,-0.209268,-1.363243,1.246395,-0.357071,1.207866,1.377868,-2.931100,4128005,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,0.971472,-0.239761,-0.403682,-0.356640,0.081373,-0.666429,-0.257329,-0.548524,-0.234434,-0.485530,...,1.010220,0.301020,0.045051,1.103620,0.808009,1.085577,1.238311,0.508195,4102000,2013
276,-0.065916,0.045787,0.281341,0.283516,0.115535,0.182101,0.241159,0.214678,-0.057329,0.396812,...,-0.350719,-0.353635,0.059508,-0.316842,-0.150983,-0.352354,-0.531866,0.483937,4102000,2014
277,-0.188872,0.376561,0.728087,0.686122,0.383313,0.579277,0.541554,0.761754,0.121163,-0.461677,...,-2.275584,-1.688396,0.028186,-2.303846,-1.762322,-2.410254,-2.485671,0.460622,4102000,2015
278,-0.690207,-0.245376,0.668283,0.250972,-0.415886,0.069608,-0.168907,0.791883,1.109221,2.184154,...,-0.116047,-0.007323,0.026981,-0.089208,0.203929,-0.103409,0.195302,0.495182,4102000,2016


In [38]:
#k-fold para achar melhor valor de alpha
model = LassoCV(cv=10, random_state=0, max_iter=10000)

In [39]:


df_out=pd.DataFrame(columns=['codigo_ibge','2018','2019','2020'])

for i in codigo_saida:
    
    X_train_menor=X_train_scaled.loc[X_train_scaled['codigo_ibge']==i].drop(columns=['codigo_ibge','year'])
    Y_train_menor=Y_train[Y_train['codigo_ibge']==i].drop(columns='codigo_ibge').T
    
    model.fit(X_train_menor, Y_train_menor)
    
    lasso_t= Lasso(alpha=model.alpha_, max_iter= 10000).fit(X_train_menor,Y_train_menor)
    
    print(f'\n Alpha usado: {model.alpha_}')
    print('Features non-zero para {}: {}'.format(i, np.sum(lasso_t.coef_ != 0)))
    print('Features com valores non-zero (ordenados pela magnitude absoluta):')

    for e in sorted (list(zip(list(X_train), lasso_t.coef_)), key = lambda e: -abs(e[1])):
        if e[1] != 0:
            print('\t{}, {:.3f}'.format(e[0], e[1]))
    
    
    X_test_menor=X_test_scaled.loc[X_test_scaled['codigo_ibge']==i].drop(columns=['codigo_ibge','year'])
   
    predicoes=lasso_t.predict(X_test_menor)

    data_saida={'codigo_ibge':i, '2018':predicoes[0], '2019':predicoes[1], '2020':predicoes[2]}
    
    data_saida=pd.DataFrame([data_saida])
    
    df_out=pd.concat([df_out,data_saida])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 49.870449904233055
Features non-zero para 4102000: 12
Features com valores non-zero (ordenados pela magnitude absoluta):
	TOA_SW_DWN_6, 438.850
	ALLSKY_SFC_UV_INDEX_2, -186.843
	GWETTOP_11, -154.533
	T2M_MIN_4, 100.429
	GWETPROF_3, -80.012
	T2M_RANGE_1, -70.582
	ALLSKY_SFC_UVA_2, -60.038
	WS50M_RANGE_12, -55.659
	ALLSKY_SFC_UV_INDEX_1, 38.803
	ALLSKY_SFC_UVB_3, -32.122
	GWETROOT_10, 28.897
	CLRSKY_SFC_PAR_TOT_4, 18.441


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 51.04341182854127
Features non-zero para 4104303: 6
Features com valores non-zero (ordenados pela magnitude absoluta):
	T2M_MIN_4, 141.237
	ALLSKY_SFC_UV_INDEX_2, -121.331
	T2MWET_5, 82.964
	month_10, 77.318
	WS10M_RANGE_9, 75.245
	ALLSKY_SFC_UVA_3, 22.285


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...


 Alpha usado: 20.757030597128043
Features non-zero para 4127502: 11
Features com valores non-zero (ordenados pela magnitude absoluta):
	T2MWET_5, 237.685
	CLRSKY_SFC_SW_DWN_8, 232.370
	ALLSKY_SFC_UVB_2, -87.539
	RH2M_2, 77.154
	ALLSKY_SFC_UV_INDEX_2, -48.726
	GWETROOT_9, 45.000
	ALLSKY_SFC_UVA_4, -26.979
	GWETPROF_9, -25.643
	ALLSKY_SFC_UV_INDEX_1, 19.430
	QV2M_2, 2.199
	T2M_3, 0.236


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 83.34386333170184
Features non-zero para 4127700: 9
Features com valores non-zero (ordenados pela magnitude absoluta):
	RH2M_1, 223.831
	ALLSKY_SFC_UV_INDEX_2, -201.615
	T2M_MIN_4, 156.723
	T2MWET_5, 104.591
	CLRSKY_SFC_SW_DWN_7, -100.529
	ALLSKY_SFC_UV_INDEX_10, 54.580
	T2MDEW_8, 42.526
	month_9, 39.457
	ALLSKY_SFC_UV_INDEX_1, 26.877


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 0.28644945612652417
Features non-zero para 4128005: 16
Features com valores non-zero (ordenados pela magnitude absoluta):
	T2M_MIN_4, 170.521
	T2MWET_5, 153.416
	ALLSKY_SFC_UV_INDEX_2, -115.241
	T2MDEW_8, 104.496
	RH2M_1, 88.566
	CLRSKY_SFC_PAR_TOT_4, 86.282
	month_9, 48.950
	T2MWET_7, -43.448
	T2M_RANGE_10, 42.460
	GWETTOP_11, -31.339
	ALLSKY_SFC_PAR_TOT_5, 22.235
	WS10M_9, -17.512
	CLRSKY_SFC_SW_DWN_7, -8.960
	ALLSKY_SFC_UVA_4, -3.014
	TOA_SW_DWN_6, -1.937
	GWETPROF_3, -0.795


In [75]:
df_out=df_out.reset_index(drop=True)

In [76]:
df_out

,codigo_ibge,2018,2019,2020
0,4102000,3493.0,2968.0,2064.0
1,4104303,3293.0,2677.0,2496.0
2,4104428,3380.0,2852.0,2500.0
3,4104808,3655.0,3655.0,3655.0
4,4104907,3690.0,3542.0,4000.0
5,4109401,2849.0,4120.0,3529.0
6,4113205,3000.0,3350.0,3000.0
7,4113700,2880.0,2880.0,2880.0
8,4113734,3211.0,3471.0,2963.0
9,4114005,3148.0,3350.0,2592.0


In [77]:
df_out.to_csv('submission.csv',index=False)